# Логистическая регрессия
На прошлой лекции: 
$$Y = \{-1, 1\}$$
- -1, 1 - классы
- $a(x)$ должен возвращать одно из двух чисел

Логистическая функция потерь
$$Q(w, X) = \frac{1}{l} \sum _{i=1} ^l log(1 + exp(-y_i<w, x_i>)) -> min_w$$

## Предсказание вероятностей:
### Кредитный скоринг
- Давать не класс (-1, 1), а вероятность положительного класса

- Модель выдает вероятность того, что клиент вернет кредит, а не просто +1, если кредит будет возвращен и -1, если не будет.
- Стратегия: выдавать кредит только клиентам с $b(x) > 0.9$ так как 10% невозвращенных кредитов - нормально, а упущенная выгода - плохо. Целевой процент определяют финансисты.

### Баннерная реклама
- $b(x)$ - вероятность, что пользователь кликнет по рекламе
- $c(x)$ - прибыль в случае клика
- $c(x)b(x)$ - нужно оптимизировать
- Нужно показывать рекламу, на которую пользователь скорее всего кликнет, и которая стоит дороже

### Прогнозирование оттока клиентов
- получать вероятность того, что клиент уйдет
- сортировать клиентов по вероятности, обзванивать и пытаться удержать (предложить скидку)
- не предлагать скидку тем, кто и так останется

### Медицинская диагностика
- давать не только диагноз, но и уверенность

### Поисковое ранжирование
- определить, насколько хорошо веб-страница соответствует запросу
- сортировать документы по релевантности запроса

## Предсказание вероятностей
Модель $b(x)$ предсказывает вероятности, если среди объектов с $b(x)=p$ доля положительных равна $p$

- если выдаем кредиты тем, у кого вероятность его вернуть 90% или больше, то невозвратов будет не больше 10%, возвратов (положительный класс) 90%

## Линейный классификатор
Возьмем линейный классификатор
$$a(x) = sign(<w,x>)$$
Обучим, например, на логистическую функцию потерь:

$$Q(w, X) = \frac{1}{l} \sum _{i=1} ^l log(1 + exp(-y_i<w, x_i>)) -> min_w$$

Можем ли использовать оценку $<w, x>$ в качестве вероятности принадлежности к классу?

Нет, так как построенная прямая $b(x)=w_1 x + w_0$ не подходит для выражения вероятности. Она может быть меньше 0 и больше 1. 

Нужно применить к скалярному произведению какую-то функцию, которая переведет выход модели на отрезок $[0,1]$ 

Можно сделать с помощью сигмоиды:

$$\sigma(<w,x>) = \frac {1}{1+exp(-<w,x>)}$$

<img src='images/sigma.png'>


Функция принимает любое вещественное число, для маленьких чисел выдает 0, для больших 1


Вместо $b(x)=w_1 x + w_0$ будет $b(x)=\sigma(w_1 x + w_0)$

## Выводы:
- Существует много задач, где лучше предсказывать вероятность класса, а не сам класс
- Можно применить сигмоиду к линейной модели, чтобы получать числа от 0 до 1
- Нужно обучать модель так, чтобы эти числа можно было интерпретировать как вероятности

## Предсказание вероятностей
Модель для оценивания вероятностей:
$$b(x)=\sigma(<w,x>)$$

Как обучать:
- если $y_i = +1$, то вероятность $\sigma(<w,x_i>) -> 1$ или $<w,x_i> -> +\infty$
- если $y_i = -1$, то вероятность $\sigma(<w,x_i>) -> 0$ или $<w,x_i> -> -\infty$

То есть задача - сделать отступы на всех объектах максимальными:
$$y_i <w,x_i> ->max_w$$

$$-\sum _{i=1} ^ l \{ [y_i=1]\sigma(<w,x_i>) + [y_i=-1] (1-\sigma(<w,x_i>))\}->min_w$$

- Суммируем по всем объектам
- Для каждого объекта смотрим, к какому классу он относится (индикатор)
- Если объект положительный, то срабатывает первый индикатор, то смотрим на сигмоиду от скалярного произведения, берём её с минусом и минимизируем
- Если объект отрицательный, то срабатывает второй индикатор, то смотрим на второе слагаемое
- Для отрицательных объектов вероятность должна быть как можно меньше
- Для положительных - как можно больше

### Недостаток формулы:
Если возникла ошибка:
- Если $y_i$ = +1 и $\sigma(<w, x_i>)=0$ (объект положительный, но модель абсолютно уверена, что объект отрицательный)
- За настолько грубую ошибку модель нужно сильно штрафовать
- Здесь штраф -1 (если $y_i=1$ и $\sigma(<w, x_i>)=1$), то срабатывает первое слагаемое. С минусом будет -1 (минимально возможное значение функции потерь). В случае же грубой ошибки функция потерь будет равна 0. То есть штраф увеличен всего на 1. 

Нужно ужесточить наказание:

$$-\sum _{i=1} ^ l \{ [y_i=1]log \sigma(<w,x_i>) + [y_i=-1] log(1-\sigma(<w,x_i>))\}->min_w$$

Если $y_i = +1$ и $\sigma(<w, x_i>)=0$, то штраф равен $-log0=+\infty$

Такая функция потерь называется __log-loss__
$L(y,z)=-[y=1] log z-[y=-1]log (1-z)$

Если преобразовать формулу, то получится 

$$\sum _{i=1} ^{l} {[y_i=1] log (1 +exp(-<w,x>)) + [y_i=-1]log(1+exp(<w,x>))}$$

$$\sum _{i=1} ^{l} log(1+exp(-y_i <w, x_i>))$$

### Вывод:
- Логистическая функция максимизирует отступы - то есть приближает вероятности к 1 или 0 в зависимости от класса
- Это не строгий вывод (почему log для штрафа, почему сигмоида, а не что-то другое)